In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'UBC-OCEAN:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F45867%2F6924515%2Fupload%2Fpublic.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240312%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240312T202455Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2bb2c47ed4456dc24c53bafd27749dd83e8aab6ff6bda18c692bfe04c5e4794d88ab3ed36ce9566ff079556ee388e4ff67b2991ebb40ae3874f83269a3795f3b0a1c09d3668aad3fddfdd82fd04f22c1a9452fd1fe89603652b9d33b68db7ca4fec3628623532bbbc615e8df60d1621958c656d9505f13fee725c4f06698d2bd659b39b95186df55c7e6b3b206d7d2b59ea2588bbc34ec4a47f040b5020f86c429f5bf411bdd16de59e8feb084468030324e360ba686c5209c2432db8a5356a4abfa713079b83263cd3747655e82d9c1c770fd93ae109ec19f53a995848500c7c55c1f5060985968eced2108e3112818b8951be0e78b289ed03994c5b74b50ee'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
from PIL import Image, ImageEnhance

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import tensorflow as tf
import cv2

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Step 1: Image Rescaling

In [ ]:
# function to preprocess images
def preprocess_images(input_folder, output_folder, target_size=(224, 224)):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through each image in the input folder
    for filename in os.listdir(input_folder):
        # Construct the input path for the current image
        input_path = os.path.join(input_folder, filename)

        # Read the image using OpenCV
        img = cv2.imread(input_path)

        # Resize the image to the target size, using a target size of 224x224
        resized_img = cv2.resize(img, target_size)


        # Construct the output path for the resized image
        output_path = os.path.join(output_folder, filename)

        # Save the resized image to the output folder
        cv2.imwrite(output_path, resized_img)

In [ ]:
input_folder = "/kaggle/input/UBC-OCEAN/train_thumbnails"
output_folder = "/kaggle/working/rescaled_images"

# Set the target size for resizing
target_size = (224, 224)

# Preprocess images
preprocess_images(input_folder, output_folder, target_size)

# 2. Data Augmentation

In [ ]:
def adjust_random_brightness(image_path):
    """
    Adjust the brightness of an image to a random factor between 0.5 and 1.5 and save the result.
    Parameters:
        - image_path: The file path of the input image.
    """
    original_image = Image.open(image_path)

    # Generate a random brightness factor between 0.5 and 1.5
    random_factor = random.uniform(0.5, 1.5)
    print(f"Adjusting brightness by a factor of {random_factor}")

    # Create a brightness enhancer and apply the random factor
    enhancer = ImageEnhance.Brightness(original_image)
    BA_image = enhancer.enhance(random_factor)

    # Create a new file name for the adjusted image
    brightness_image_path = os.path.join(folder_path, f"{file_name}_brightness_altered")


    # Save the adjusted image
    BA_image.save(brightness_image_path)
    print(f"Brightness adjusted image saved: {brightness_image_path}")


def adjust_contrast(image_path):
    """
    Adjust the contrast of an image to a random factor between 0.5 and 1.5 and save the result.
    Parameters:
        - image_path: The file path of the input image.
    """
    original_image = Image.open(image_path)

    # Generate a random contrast factor between 0.5 and 1.5
    random_factor = random.uniform(0.5, 1.5)
    print(f"Adjusting contrast by a factor of {random_factor}")

    # Create a contrast enhancer and apply the random factor
    enhancer = ImageEnhance.Contrast(original_image)
    contrast_image = enhancer.enhance(random_factor)

    # Create a new file name for the adjusted image
    contrast_image_path = os.path.join(folder_path, f"{file_name}_contrast_altered")

    # Save the adjusted image
    contrast_image.save(contrast_image_path)
    print(f"Contrast adjusted image saved: {contrast_image_path}")


def random_crop(image):
    '''
    Cropping the image in the center from a random margin from the borders
    '''
    margin = 1 / 3.5
    width, height = image.size
    start_x = int(random.uniform(0, width * margin))
    start_y = int(random.uniform(0, height * margin))
    end_x = int(random.uniform(width * (1 - margin), width))
    end_y = int(random.uniform(height * (1 - margin), height))

    cropped_image = image.crop((start_x, start_y, end_x, end_y))
    return cropped_image


def crop_images(image_path):
    """
    Crop the given image and save the result.
    Parameters:
        - image_path: The file path of the input image.
    """
    original_image = Image.open(image_path)

    # Apply random cropping
    cropped_image = random_crop(original_image)

    # Create a new file name for the cropped image
    cropped_image_path = os.path.join(folder_path, f"{file_name}_cropped")

    # Save the cropped image
    cropped_image.save(cropped_image_path)
    print(f"Cropped image saved: {cropped_image_path}")


def horizontal_flip_images(image_path):
    """
    Perform horizontal flipping on the given image and save the result.
    Parameters:
        - image_path: The file path of the input image.
    """
    original_image = Image.open(image_path)

    # Perform horizontal flipping
    flipped_image = original_image.transpose(Image.FLIP_LEFT_RIGHT)

    # Create a new file name for the flipped image
    flipped_image_path = os.path.join(folder_path, f"{file_name}_hflipped")

    # Save the flipped image
    flipped_image.save(flipped_image_path)
    print(f"Horizontal flipped image saved: {flipped_image_path}")


def vertical_flip_images(image_path):
    """
    Perform vertical flipping on the given image and save the result.
    Parameters:
        - image_path: The file path of the input image.
    """
    original_image = Image.open(image_path)

    # Perform vertical flipping
    flipped_image = original_image.transpose(Image.FLIP_TOP_BOTTOM)

    # Create a new file name for the flipped image
    flipped_image_path = os.path.join(folder_path, f"{file_name}_vflipped")
    # Save the flipped image
    flipped_image.save(flipped_image_path)
    print(f"Vertical flipped image saved: {flipped_image_path}")


def add_random_gaussian_noise(folder_path):
    """
    Add random Gaussian noise to an image and save the result.

    Parameters:
    - image_path: The file path of the input image.
    - output_path: The file path for the output image.
    - min_sigma: Minimum standard deviation of the Gaussian noise.
    - max_sigma: Maximum standard deviation of the Gaussian noise.
    """


    # Open the image using Pillow
    original_image = Image.open(image_path)

    #Adding noise
    image_array = np.array(original_image)
    # Ensure image is in float format
    image_array = image_array.astype(float)

    # Generate random Gaussian noise
    sigma = random.uniform(50, 100)
    noise = np.random.normal(0, sigma, image_array.shape)

    # Add the noise to the image
    noisy_image = image_array + noise

    # Ensure values remain within the valid range [0, 255]
    noisy_image = np.clip(noisy_image, 0, 255)

    # Convert back to an image
    noisy_image = noisy_image.astype(np.uint8)
    noisy_image_pil = Image.fromarray(noisy_image)


    # Create a new file name for the adjusted image
    noisy_path = os.path.join(folder_path, f"{file_name}_noisy")

    # Save the noisy image
    noisy_image_pil.save(noisy_path)
    print(f"Added Gaussian noise with sigma = {sigma}. Saved to {noisy_path}")



In [ ]:
#Combined function

def augment(folder_path):
    """
    Combine all augmentation functions and apply them to images in the specified folder.
    Parameters:
        - folder_path: The path to the folder containing images to be augmented.
    """
    file_list = os.listdir(folder_path)

    # Iterate through each file in the folder
    for file_name in file_list:
        # Check if the file is an image
        if file_name.lower().endswith('.png'):
            # Create the full path to the image file
            image_path = os.path.join(folder_path, file_name)

            # Apply all augmentation functions
            adjust_random_brightness(image_path)
            adjust_contrast(image_path)
            crop_images(image_path)
            horizontal_flip_images(image_path)
            vertical_flip_images(image_path)
            add_random_gaussian_noise(image_path)

In [ ]:
#Calling the function
folder_path = '/kaggle/working/rescaled_images' #after scaling, images saved here
augment(folder_path)

#3. Creating validation images directory

In [ ]:
# Set the paths
valid_images_path = '/kaggle/output/valid_images'

# Create the validation folder if it doesn't exist
os.makedirs(valid_images_path, exist_ok=True)

# Get the list of all images in the train folder
all_images = os.listdir(train_images_path)

# Calculate the number of images to move (25%)
num_images_to_move = int(0.25 * len(all_images))

# Randomly select the images to move
images_to_move = random.sample(all_images, num_images_to_move)

# Move the selected images to the validation folder
for image in images_to_move:
    src_path = os.path.join(train_images_path, image)
    dest_path = os.path.join(valid_images_path, image)
    shutil.move(src_path, dest_path)

#4. Dataloader

In [ ]:
#After scaling and augmenting the data, we need to make sure the new images have their original unique numerical id + "_flipped"
# The following class makes sure that pytorch can process the dataset during training, including accessing labels

class CustomDataset(Dataset):
    def __init__(self, root_dir, csv_file):
        self.root_dir = root_dir
        self.df = pd.read_csv(csv_file)
        self.mapping_dict = self.create_mapping_dict()

    def create_mapping_dict(self):
        mapping_dict = {}
        for idx in range(len(self.df)):
            numeric_id = str(self.df.iloc[idx, 0])  # Assuming the numeric ID is in the first column
            for suffix in ['blurred', 'noisy', 'hflipped', 'vflipped', 'cropped']:
                augmented_id = f"{numeric_id}_{suffix}"
                mapping_dict[augmented_id] = numeric_id
        return mapping_dict

    def __len__(self):
        # Count the total number of augmented images (5 times the number of original images)
        return len(self.mapping_dict)

    def __getitem__(self, idx):
        augmented_id = list(self.mapping_dict.keys())[idx]
        numeric_id = self.mapping_dict[augmented_id]

        img_path = os.path.join(self.root_dir, f"{numeric_id}_{augmented_id.split('_')[1]}.png")
        image = Image.open(img_path)

        label = self.df.loc[self.df['numeric_id'] == int(numeric_id), 'label'].values[0]

        return image, label


In [ ]:
#dataset and dataloaders
training_dataset = CustomDataset(csv_file='/kaggle/input/UBC-OCEAN/train.csv', root_dir='/kaggle/input/UBC-OCEAN/train_images')
train_DL = DataLoader(training_dataset, batch_size=32, shuffle=True)

#repeat for validation
validation_dataset = CustomDataset(csv_file='/kaggle/input/UBC-OCEAN/train.csv', root_dir='/kaggle/input/UBC-OCEAN/valid_images')
validation_DL = DataLoader(validation_dataset, batch_size=32, shuffle=True)


#5. Training Loop

In [ ]:
# TODO: Build and train your network
model = models.vgg16(pretrained = True) #Loading pre-trained network

#Freeze parameters to avoid backpropagation
for param in model.parameters():
    param.requires_grad = False


model = model.to('cuda')

#Defining new untrained feed-forward network
classifier = nn.Sequential(nn.Linear(25088,4096),
                          nn.ReLU(), #activation function - ReLU is effective, computationally inexpensive
                                     #and removes the vanishing gradient problem
                          nn.Dropout(0.2),
                          nn.Linear(4096, 256),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(256, 64),
                          nn.Dropout(0.2), #Removed 20% of data each time, good place to start
                          nn.Linear(64, 6),  #Must be 6 because 6 = number of classes
                          nn.LogSoftmax(dim=1))


classifier = classifier.to('cuda')
model.classifier = classifier
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr = 0.01)

#Training
epochs = 5 #maybe change to 10 later on? depending on output
train_loss = 0

for epoch in range(epochs):
    #model.train()
    for inputs, labels in train_DL:
      model.train()
      inputs, labels = inputs.to('cuda'), labels.to('cuda')
      optimizer.zero_grad()
      outputs = model.forward(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      train_loss += loss.item()


    model.eval()
    with torch.inference_mode():
      validation_loss = 0
      accuracy = 0
      for inputs, labels in validation_DL:
          inputs, labels = inputs.to('cuda'), labels.to('cuda')
          outputs = model.forward(inputs)
          running_valid_loss = criterion(outputs, labels).item()
          validation_loss += running_valid_loss

          ps = torch.exp(outputs)
          top_p, top_class = ps.topk(1, dim = 1)
          equals = top_class == labels.view(*top_class.shape)
          accuracy += torch.mean(equals.type(torch.FloatTensor))

    print(f"Epoch {epoch+1}/{epochs}...",
          f"Train loss: {train_loss/len(train_DL):.3f}..."
        f"Validation loss: {validation_loss/len(validation_DL):.3f}..."
        f"Validation accuracy: {accuracy:.3f}...")
    train_loss = 0

